In [1]:
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
# from pykrige.rk import RegressionKriging
import geopandas as gpd
import rasterio
import rioxarray
import pdal

In [2]:
# load the bathy points

pts_gdf_all = gpd.read_file('../data/test_sites/florida_keys/all_bathy_pts.gpkg')


In [3]:
pts_gdf_all

,X,Y,Z,Z_g,geoid_corr,tide_ocean_corr,geof2m_corr,delta_time,oc_sig_conf,land_sig_conf,...,Z_refr,easting_corr,northing_corr,z_kde,kde_val,atm_profile,beamtype,oc_hconf_perc,fema_elev,geometry
0,-80.888159,24.701906,-36.095867,-11.829935,-24.463730,0.136300,0.061499,2018-10-17T08:53:09.661000,1,4,...,-8.838998,-0.004978,-0.005051,-8.434565,0.161033,profile_2,weak,95.336422,-8.239627,POINT (511312.965 2731945.463)
1,-80.888156,24.701925,-36.415703,-12.149771,-24.463730,0.136300,0.061499,2018-10-17T08:53:09.661000,1,4,...,-9.077264,-0.005114,-0.005188,-8.434565,0.162915,profile_2,weak,95.336422,-8.189320,POINT (511313.171 2731947.597)
2,-80.888153,24.701957,-31.863705,-7.597773,-24.463730,0.136300,0.061499,2018-10-17T08:53:09.661000,1,4,...,-5.686029,-0.003182,-0.003228,-8.434565,0.164943,profile_2,weak,95.336422,-8.125921,POINT (511313.513 2731951.153)
3,-80.888153,24.701957,-36.333305,-12.067373,-24.463730,0.136300,0.061499,2018-10-17T08:53:09.661000,1,4,...,-9.019644,-0.005073,-0.005147,-8.434565,0.165691,profile_2,weak,95.336422,-8.125921,POINT (511313.515 2731951.156)
4,-80.888148,24.702002,-38.159710,-13.893675,-24.463814,0.136280,0.061498,2018-10-17T08:53:09.660000,1,4,...,-10.385779,-0.005838,-0.005925,-8.408025,0.168430,profile_2,weak,95.336422,-8.117629,POINT (511313.999 2731956.139)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391516,-80.920901,24.775501,-28.404345,-4.047483,-24.432243,0.014260,0.061124,2022-01-10T00:31:00.684000,1,4,...,-3.018985,0.010330,-0.005289,-2.164090,0.178308,profile_3,weak,96.932672,-2.333178,POINT (507996.286 2740091.983)
391517,-80.920894,24.775565,-25.838154,-1.481199,-24.432438,0.014360,0.061123,2022-01-10T00:31:00.683000,1,4,...,-1.105579,0.003773,-0.001932,-2.164090,0.178430,profile_3,weak,96.932672,-2.189999,POINT (507996.970 2740099.055)
391518,-80.920890,24.775610,-26.457558,-2.100603,-24.432438,0.014360,0.061123,2022-01-10T00:31:00.682000,1,4,...,-1.567559,0.005354,-0.002741,-2.164090,0.178339,profile_3,weak,96.932672,-2.318623,POINT (507997.441 2740104.009)
391519,-80.920884,24.775661,-27.968256,-3.611301,-24.432438,0.014360,0.061123,2022-01-10T00:31:00.681000,1,4,...,-2.694308,0.009210,-0.004716,-2.164090,0.178838,profile_3,weak,96.932672,-2.388885,POINT (507997.975 2740109.672)


In [4]:
# # create a numpy array that PDAL can read by subsetting the columns and renaming the northing, easting, etc columns to the LAS defaults (Z,Y,Z)
pdal_array = pts_gdf_all.loc[:,['northing','easting','z_kde']].rename(columns={'northing':'Y','easting':'X','z_kde':'Z'}).to_records(index=False)

# pipeline = pdal.Filter.elm(cell=5).pipeline(pdal_array)
# pipeline |= pdal.Writer.las(filename='../data/resample_test/all_pts.laz')

# print(pipeline.execute())

In [5]:

# create a PDAL pipeline
pipeline = pdal.Filter.relaxationdartthrowing(count=8000).pipeline(pdal_array)

print(pipeline.execute())

thinned_array = pipeline.arrays[0]

pipeline = pdal.Writer.las(filename='../data/resample_test/filtered.laz').pipeline(thinned_array)
print(pipeline.execute())

8000
8000


In [6]:
pts_gdf = gpd.GeoDataFrame(thinned_array,geometry=gpd.points_from_xy(thinned_array['X'],thinned_array['Y'],crs='EPSG:32617'))

In [7]:
# keep points within a half meter of the estimated seafloor
# pts_gdf = pts_gdf[abs(pts_gdf.Z_g - pts_gdf.z_kde) < 0.005]

In [8]:
def krige_bathy(pts_gdf,krmodel):
    # pts_gdf = pts_gdf.sort_values('kde_val',ascending=False).iloc[:3000]
    ras = rasterio.open('../data/resample_test/bilinear.tif')
    ar = rioxarray.open_rasterio(ras)
    gridx = ar.x.data
    gridy = ar.y.data
    # going to randomly downsample the lidar point dataframe to make the UK model faster

    # pts_gdf = pts_gdf.sample(1000)
    pts_gdf.to_file('../data/resample_test/keys_testpts.gpkg')

    x_loc = pts_gdf.geometry.x.to_numpy()
    y_loc = pts_gdf.geometry.y.to_numpy()
    z_elev = pts_gdf.Z.to_numpy()
    krigemodel = krmodel(
        x=x_loc,
        y=y_loc,
        z=z_elev,
        variogram_model='linear',
        verbose=True,
        coordinates_type = "euclidean",
    )

    # krigemodel = UniversalKriging(
    #     x=x_loc,
    #     y=y_loc,
    #     z=z_elev,
    #     variogram_model='linear',
    #     verbose=True,
    #     # coordinates_type = "euclidean",
    # )
        
    z,ss = krigemodel.execute('grid',gridx,gridy)
    # xarray/rioxarray are too complicated, going to just use rasterio

    with rasterio.open('../data/resample_test/interp_OK.tif',mode='w+',crs=ras.crs,width=ras.width,height=ras.height,count=2,dtype=ras.dtypes[0],transform=ras.transform,) as rasout:
        rasout.write(z,1)
        rasout.write(ss,2)


krige_bathy(pts_gdf=pts_gdf,krmodel=OrdinaryKriging)

/home/mlinds/mambaforge/envs/thesis_env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


Adjusting data for anisotropy...
Initializing variogram model...
Coordinates type: 'euclidean' 

Using 'linear' Variogram Model
Slope: 0.0001867234643833979
Nugget: 1.4260770798121374 

Calculating statistics on variogram model fit...
Executing Ordinary Kriging...



: 

: 